# 📏 Exercício: Predição Conforme (Conformal Prediction) para Regressão

Neste exercício, você vai implementar o método de **Conformal Prediction** para prever intervalos de confiança em tarefas de regressão. Vamos usar um modelo simples de regressão (como Regressão Linear) e aplicar **Conformal Prediction com split**.

---

## 🎯 Objetivo

Prever não apenas um valor pontual para $Y$, mas um **intervalo** $[L(x), U(x)]$ que contenha o valor verdadeiro com uma probabilidade próxima de $1 - \alpha$, mesmo sem fazer suposições fortes sobre a distribuição dos erros.

---

## 📂 Passo 1 — Carregando os dados

Vamos usar um conjunto de dados sintético:

    import numpy as np
    from sklearn.datasets import make_regression
    import matplotlib.pyplot as plt

    X, y = make_regression(n_samples=300, n_features=1, noise=15, random_state=42)

---

## ✂️ Passo 2 — Divisão treino / calibração / teste

Dividimos os dados em três partes:

- **Treino**: para ajustar o modelo  
- **Calibração**: para estimar os resíduos e ajustar os quantis  
- **Teste**: para avaliar a cobertura e largura dos intervalos

    from sklearn.model_selection import train_test_split

    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    X_train, X_calib, y_train, y_calib = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=0)

---

## 🧠 Passo 3 — Ajuste do modelo de regressão

    from sklearn.linear_model import LinearRegression

    model = LinearRegression()
    model.fit(X_train, y_train)

---

## 📏 Passo 4 — Predição conforme (Split Conformal Prediction)

### 🌟 Ideia-chave

Queremos construir intervalos $[ \hat{y}(x) - q, \hat{y}(x) + q ]$ tais que a chance de conter o verdadeiro valor $y$ seja de pelo menos $1 - \alpha$, **sem assumir distribuição do erro**.

### 📐 Etapas:

1. Usar o conjunto de calibração para calcular os **resíduos absolutos**:

   $$
   r_i = |y_i - \hat{y}(x_i)|
   $$

2. Calcular o **quantil ajustado**:

   $$
   q = \text{quantil}_{1 - \alpha}(r_1, \ldots, r_n)
   $$

   ou, mais corretamente:

   $$
   q = \text{quantil}_{\lceil (1 - \alpha)(n + 1) \rceil / n}(r_1, \ldots, r_n)
   $$

3. Construir intervalo para cada novo ponto:

   $$
   [ \hat{y}(x) - q, \hat{y}(x) + q ]
   $$

---

## 🧮 Passo 5 — Calculando o quantil ajustado

    from sklearn.metrics import mean_absolute_error

    y_calib_pred = model.predict(X_calib)
    residuals = np.abs(y_calib - y_calib_pred)

    alpha = 0.1
    n_calib = len(residuals)
    q = np.quantile(residuals, np.ceil((1 - alpha) * (n_calib + 1)) / n_calib)

---

## 🔍 Passo 6 — Fazendo predições e construindo intervalos

    y_test_pred = model.predict(X_test)
    lower_bounds = y_test_pred - q
    upper_bounds = y_test_pred + q

---

## 📊 Passo 7 — Avaliando a cobertura


Avalie a cobertura empirica média dos intervalos, isto é, quantas vezes a previsão real está dentro do intervalo conforme.

---

## 🖼️ Passo 8 — Visualizando os intervalos

Crie uma visualização com o valor real vs valor predito e o intervalo conforme. 

---

## ✅ Conclusão

- O método **Split Conformal Prediction** fornece intervalos **válidos** com **garantia de cobertura** marginal.  
- Não assume nada sobre a distribuição dos resíduos.  
- O uso do **quantil ajustado** com calibração é o truque fundamental para garantir validade.

Você pode experimentar com outros modelos (por exemplo, Random Forest ou SVR) e ver como isso afeta a largura dos intervalos e a cobertura.
